# Config stuff

In [1]:
import ConnectionConfig as cc
from delta import DeltaTable
cc.setupEnvironment()

# Start local cluster

In [2]:
spark = cc.startLocalCluster("FACT_RIDES")
spark.getActiveSession()

# Create facts table: rides

## Read from sources

### Read from VeloDB database

Filteren en joinen in de brontabel of in de spark omgeving? Momenteel gekozen om de rides table en de locks en stations table apart in te lezen en ze later te joinen in de spark omgeving. Zorgt voor minder DB operaties.

In [3]:
#EXTRACT

cc.set_connectionProfile("VeloDB")

# Read rides table from source VeloDB database
rides_source_df = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable", "(select rideid, starttime, endtime, subscriptionid, startlockid, endlockid from rides) as subq") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "rideid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 4140000) \
    .load()

# Next read operation performs a join between the locks and stations source tables. The goal is to retrieve a zipcode for each lockid. This zipcode is used to link the weather dimension with the facts table 
locks_with_zip = spark.read \
    .format("jdbc") \
    .option("url", cc.create_jdbc()) \
    .option("driver" , cc.get_Property("driver")) \
    .option("dbtable", "(select l.lockid, s.zipcode from locks l \
    left outer join stations s on l.stationid = s.stationid) as subq") \
    .option("user", cc.get_Property("username")) \
    .option("password", cc.get_Property("password")) \
    .option("partitionColumn", "lockid") \
    .option("numPartitions", 4) \
    .option("lowerBound", 0) \
    .option("upperBound", 8000) \
    .load()

locks_with_zip.show(10)

+------+-------+
|lockid|zipcode|
+------+-------+
|     1|   2000|
|     2|   2000|
|     3|   2000|
|     4|   2000|
|     5|   2000|
|     6|   2000|
|     7|   2000|
|     8|   2000|
|     9|   2000|
|    10|   2000|
+------+-------+
only showing top 10 rows



In [4]:
rides_source_df.show(10)

+------+-------------------+-------------------+--------------+-----------+---------+
|rideid|          starttime|            endtime|subscriptionid|startlockid|endlockid|
+------+-------------------+-------------------+--------------+-----------+---------+
|     1|2015-09-22 00:00:00|2012-09-22 00:00:00|         13296|       4849|     3188|
|     2|2015-09-22 00:00:00|2012-09-22 00:00:00|         45924|       NULL|     NULL|
|     3|2015-09-22 00:00:00|2012-09-22 00:00:00|         25722|       2046|     1951|
|     4|2015-09-22 00:00:00|2012-09-22 00:00:00|         31000|       1821|     2186|
|     5|2015-09-22 00:00:00|2012-09-22 00:00:00|         59732|       6382|     2700|
|     6|2015-09-22 00:00:00|2012-09-22 00:00:00|          NULL|       NULL|     NULL|
|     7|2015-09-22 00:00:00|2012-09-22 00:00:00|         31055|       1388|     3401|
|     8|2015-09-22 00:00:00|2012-09-22 00:00:00|         65164|       2572|       13|
|     9|2015-09-22 00:00:00|2012-09-22 00:00:00|      

### Read from deltatables

In [4]:
#EXTRACT
# Dimension date
dim_date = spark.read.format("delta").load("spark-warehouse/dimdate")

# Dimension weather
dim_weather = spark.read.format("delta").load("spark-warehouse/dimweather")

### Read from weather data source

In [50]:
#EXTRACT
weather_responses = spark.read.format("json").option("multiLine",True).load("weather")
weather_responses.show(10)

+--------+------+---+--------------+----------+-------+--------------------+-----+------+--------------------+--------+----------+--------------------+-----------------+-------+
|    base|clouds|cod|         coord|        dt|     id|                main| name|  rain|                 sys|timezone|visibility|             weather|             wind|zipCode|
+--------+------+---+--------------+----------+-------+--------------------+-----+------+--------------------+--------+----------+--------------------+-----------------+-------+
|stations| {100}|200|{44.34, 10.99}|1583593967|3163858|{298.74, 933, 64,...|Zocca|{3.16}|{IT, 2075663, 166...|    7200|     10000|[{overcast clouds...|{349, 1.18, 0.62}|   2000|
|stations| {100}|200|{44.34, 10.99}|1583132121|3163858|{298.74, 933, 64,...|Zocca|{3.16}|{IT, 2075663, 166...|    7200|     10000|[{overcast clouds...|{349, 1.18, 0.62}|   2018|
|stations| {100}|200|{44.34, 10.99}|1583134645|3163858|{298.74, 933, 64,...|Zocca|{3.16}|{IT, 2075663, 166...|

## Create tempviews

In [51]:
# Rides source table
rides_source_df.createOrReplaceTempView("ridesSource")

# Table with lockid's and zipcodes
locks_with_zip.createOrReplaceTempView("locksZip")

# Dimension date
dim_date.createOrReplaceTempView("dimDate")

# Dimension weather
dim_weather.createOrReplaceTempView("dimWeather")

# Weather responses
weather_responses.createOrReplaceTempView("weatherResponses")

## Add zipcodes to rides table

Join ridesSource with locksZip on lockid so we can add the zipcode to the rides table

In [7]:
# TRANSFORM
rides_with_zipcodes = spark.sql("select src.rideid, src.starttime, src.endtime, src.subscriptionid, src.startlockid, \
                                 src.endlockid, lz.zipcode as startlockZipcode \
                                 from ridesSource as src \
                                 left outer join locksZip as lz on src.startlockid = lz.lockid")
rides_with_zipcodes.show(10)

+------+-------------------+-------------------+--------------+-----------+---------+----------------+
|rideid|          starttime|            endtime|subscriptionid|startlockid|endlockid|startlockZipcode|
+------+-------------------+-------------------+--------------+-----------+---------+----------------+
|     4|2015-09-22 00:00:00|2012-09-22 00:00:00|         31000|       1821|     2186|            2018|
|     9|2015-09-22 00:00:00|2012-09-22 00:00:00|         71164|         50|     2067|            2000|
|     1|2015-09-22 00:00:00|2012-09-22 00:00:00|         13296|       4849|     3188|            2140|
|     5|2015-09-22 00:00:00|2012-09-22 00:00:00|         59732|       6382|     2700|            2660|
|    11|2015-09-22 00:00:00|2012-09-22 00:00:00|           999|        985|     2148|            2018|
|     2|2015-09-22 00:00:00|2012-09-22 00:00:00|         45924|       NULL|     NULL|            NULL|
|     3|2015-09-22 00:00:00|2012-09-22 00:00:00|         25722|       204

In [8]:
rides_with_zipcodes.createOrReplaceTempView("ridesWithZip")

## Transform weather responses table

Add weather_SK based on the weather type (condition_id)
Condition id:
- < 800: All codes with a number smaller than 800 means rain in some form. (=onaangenaam code 2)
- = 800: This code means clear sky and sunshine (=aangenaam code 1 if temperature is higher than 15 degrees Celsius)
- \> 800: All other weather conditions (Neutraal code 3)

In [52]:
#TRANSFORM
short_weather_responses = spark.sql("select zipCode as zip_code, dt as timestamp, weather.id[0] as condition_id, main.temp as temperature, \
                                    case \
                                        when condition_id < 800 then 2 \
                                        when condition_id = 800 and main.temp > (273 + 15) then 1 \
                                        when condition_id = 800 and main.temp < (273 + 15) then 3 \
                                        when condition_id > 800 then 3 \
                                    else 4 \
                                    end as weather_SK \
                                    from weatherResponses")
short_weather_responses.show(10)

+--------+----------+------------+-----------+----------+
|zip_code| timestamp|condition_id|temperature|weather_SK|
+--------+----------+------------+-----------+----------+
|    2000|1583593967|         804|     298.48|         3|
|    2018|1583132121|         804|     298.48|         3|
|    2020|1583134645|         804|     298.48|         3|
|    2030|1585973548|         804|     298.48|         3|
|    2050|1569163778|         804|     298.48|         3|
|    2060|1569968446|         804|     298.48|         3|
|    2100|1575264035|         804|     298.48|         3|
|    2140|1619936979|         804|     298.48|         3|
|    2170|1656838375|         804|     298.48|         3|
|    2600|1659778730|         804|     298.48|         3|
+--------+----------+------------+-----------+----------+
only showing top 10 rows



In [53]:
short_weather_responses.createOrReplaceTempView("shortWeatherResponses")

## Build facts table

Koppeling met date dimensie:
- Moet endtime ook een SK hebben naar de date dimensie? Want zorgt voor extra kolom --> Niet goed in feitentabel!
- Eventueel van de kolom date_SK de koppeling naar starttime en endttime samenvoegen?

In [61]:
#TRANSFORM
rides_fact_df = spark.sql("select src.rideid as ride_ID, dd.date_SK, \
                          coalesce(wr.weather_SK, 4) as weather_SK \
                          from ridesWithZip as src \
                          left outer join dimDate as dd \
                          on cast(src.starttime as DATE) = cast(dd.CalendarDate as DATE) \
                          left outer join shortWeatherResponses wr \
                          on src.startlockZipcode = wr.zip_code \
                          and date_format(src.starttime, 'yyyy-MM-dd-HH') = date_format(from_unixtime(wr.timestamp),'yyyy-MM-dd-HH') \
                          where src.subscriptionid is not null ")

rides_fact_df.show(10)

+-------+-------+----------+
|ride_ID|date_SK|weather_SK|
+-------+-------+----------+
|      4|   1096|         4|
|      9|   1096|         4|
|     12|   1096|         4|
|      1|   1096|         4|
|      5|   1096|         4|
|     11|   1096|         4|
|      2|   1096|         4|
|      3|   1096|         4|
|      7|   1096|         4|
|      8|   1096|         4|
+-------+-------+----------+
only showing top 10 rows



### Create temview from facts table

In [57]:
rides_fact_df.createOrReplaceTempView("factRides")

## Write facts table to delta table

In [58]:
# LOAD
rides_fact_df.write.format("delta").mode("overwrite").saveAsTable("factRides")